In [3]:
%load_ext autoreload
%autoreload 

import os 
import shutil
import numpy as n
from datetime import date
from matplotlib import pyplot as plt
import time
import napari

In [4]:
os.chdir('/home/ali/packages/s2p-lbm/')

from suite3d.job import Job
from suite3d import lbmio, utils, ui, nbtools
from suite3d.io import get_tif_tag
from suite3d import tiff_utils as tfu
from suite3d import file_utils as flu

/home/ali/anaconda3/envs/suite3d-gpu/lib/python3.8/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/home/ali/anaconda3/envs/suite3d-gpu/lib/python3.8/site-packages/paramiko/transport.py:258: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [5]:
subjects_dir =   '/mnt/zortex-subjects/'
expt_info = { 
    'subject':         'SS003',
    'date' :          '2024-08-15',
    'expnum' :         [1,2,3,4],}

tifs, si_params, exp_str = flu.find_exp(subjects_dir, **expt_info, verbose=False, get_si_params=False)
vol_rate = get_tif_tag(tifs[0],'SI.hRoiManager.scanVolumeRate')

In [6]:
subjects_dir =   '/mnt/zortex-subjects/'
expt_info = { 
    'subject':         'SS003',
    'date' :          '2024-08-15',
    'expnum' :         [1,2,3,4],}

original_tifs, si_params, exp_str = flu.find_exp(subjects_dir, **expt_info, verbose=False, get_si_params=False)
fixed_tif_dir = '/mnt/md0/runs/s3d-SS003_2024-08-15_1-2-3-4/raw'
tifs = n.concatenate([tfu.get_tif_paths(os.path.join(fixed_tif_dir,str(dir))) for dir in expt_info['expnum']])
vol_rate = get_tif_tag(original_tifs[0],'SI.hRoiManager.scanVolumeRate')

job_params = {    
    # number of channels recorded in the tiff file, typically 30
    'n_ch_tif' : 7,
    # convert from the Scanimage channel ordering to deep-to-shallow ordering
    # in our case, ScanImage channel numbers are in temporal order. 
    'planes' : n.array([ 0, 1,2,3,4,5,6]),
    # number of files to use for the initial pass
    'n_init_files' :   1,

    # volume rate in acquisition
    'fs' : vol_rate,
    'lbm' : False,
    'subtract_crosstalk' : False,
    'fuse_strips' : False,
    
    # 3D GPU registration - fast! 
    # for now, 3D + GPU are well-tested, other options might have minor bugs
    # if you run into them, let us know! we'll fix them
    '3d_reg' : False,
    'gpu_reg' : True,
}


savedir = os.path.join('/mnt/md0/runs', expt_info['subject'], expt_info['date'])
os.makedirs(savedir, exist_ok=True)

# Create the job
job = Job('/mnt/md0/runs',exp_str, tifs = tifs,
          params=job_params, create=False, overwrite=False, verbosity = 3)

   Found and loaded params from /mnt/md0/runs/s3d-SS003_2024-08-15_1-2-3-4/params.npy


/tmp/ipykernel_240409/756748267.py:9: DeprecationWarning: Function get_tif_paths is deprecated. Refactor: use the one in suite3d.io.tiff_utils instead.
  tifs = n.concatenate([tfu.get_tif_paths(os.path.join(fixed_tif_dir,str(dir))) for dir in expt_info['expnum']])


In [7]:
img3d = job.load_summary()['ref_img_3d']

In [11]:
corr_map = job.load_corr_map_results()['vmap']

In [8]:
import k3d

In [13]:
from k3d.colormaps import matplotlib_color_maps

In [18]:
seg = job.load_segmentation_results()
shape = seg['info']['vmap'].shape
cell_id_vol, rgb_vol = ui.make_label_vols(seg['stats'], shape, lam_max=0.25, cmap='Set1',)

      Loading from /mnt/md0/runs/s3d-SS003_2024-08-15_1-2-3-4/rois/info.npy
      Loading from /mnt/md0/runs/s3d-SS003_2024-08-15_1-2-3-4/rois/stats.npy
      Loading from /mnt/md0/runs/s3d-SS003_2024-08-15_1-2-3-4/rois/iscell.npy


In [28]:
plot = k3d.plot()
plt_corrmap = k3d.marching_cubes(rgb_vol[1:, :,:,3], level = 0.5, color=0xe31b23)
plt_mip = k3d.mip(corr_map[1:, 200:400, 200:400], color_map = matplotlib_color_maps.Greys, color_range = [0,5],)
plot += plt_corrmap
plot += plt_mip
plot.display()

Output()

In [16]:
# plt_mip = k3d.mip(corr_map[1:, 200:400, 200:400], color_map = matplotlib_color_maps.Greys, color_range = [0,5],)
# plot = k3d.plot()
# plot += plt_mip
# plot.display()

Output()